# Data Preparation

## Necessary imports

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [3]:
! pwd
! ls -la
! head -n 3 amazon_item_ratings.csv
! tail -n 3 amazon_item_ratings.csv

/home/big/ABD-Projeto
total 673856
drwxrwxr-x  8 big big      4096 mai 23 19:59  .
drwxr-xr-x 28 big big      4096 mai 23 19:49  ..
drwxrwxr-x  5 big big      4096 mai 23 19:49  ALSmodel
-rw-rw-r--  1 big big 689932433 mai 18 20:43  amazon_item_ratings.csv
-rw-rw-r--  1 big big     16978 mai 23 19:49 'Data Preparation.ipynb'
-rw-rw-r--  1 big big       684 mai 23 19:49  derby.log
drwxrwxr-x  8 big big      4096 mai 23 19:49  .git
drwxrwxr-x  2 big big      4096 mai 23 19:49  .ipynb_checkpoints
drwxrwxr-x  2 big big      4096 mai 23 19:49  items.parquet
drwxrwxr-x  4 big big      4096 mai 23 19:49  metastore_db
-rw-rw-r--  1 big big     13090 mai 23 19:49  Pre-Computing.ipynb
-rw-rw-r--  1 big big        12 mai 23 19:49  README.md
-rw-rw-r--  1 big big      9347 mai 23 19:49  Results.ipynb
drwxrwxr-x  5 big big      4096 mai 23 19:49  spark-warehouse
A3AF8FFZAZYNE5,0000000078,5.0,1092182400
A2X4DOBWXXTX4A,1615725415,5.0,1327536000
A202HM75ZHSEGJ,B0059XTUB8,5.0,1335830400
A2V1XSFJL9BI3,0

## Read dataset from file

In [4]:
df_items = spark.read.csv("amazon_item_ratings.csv", header=False, inferSchema=True, sep=",") 

Get a fraction of the sample

In [5]:
df_items = df_items.sample(fraction=0.05) #0.05 works

## Multiple checks on structure

Check dataset schema and column datatypes

In [6]:
df_items.printSchema()
df_items.count()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: integer (nullable = true)



826344

Display 10 rows

In [7]:
df_items.show(n=10, truncate=False)

+--------------+----------+---+----------+
|_c0           |_c1       |_c2|_c3       |
+--------------+----------+---+----------+
|A1NHVWB3UKEBNN|0143144723|4.0|1271376000|
|A4ZKY17VUBPZH |B005X9LJBM|5.0|1386028800|
|A1FTQLG098SXR9|B007YA15B2|5.0|1342310400|
|A3QLVWSY7DFVJ4|1497408180|4.0|1404864000|
|AM8D6ZN0IRO8K |B0077RUQJ4|5.0|1381017600|
|A1CULHD7XSO6AZ|B0069G5Z9K|4.0|1360454400|
|A12JBHOSYHW03 |B006Z6E8AG|5.0|1363305600|
|A2F7F1VJZTZF0P|B003WETVKG|2.0|1305676800|
|A2RLLV430OWYJC|B001GNCM2O|5.0|1250380800|
|A1PFZN9J7XORUT|0062085484|3.0|1367280000|
+--------------+----------+---+----------+
only showing top 10 rows



Change column names to improve readabilty

In [8]:
df_items = df_items.withColumnRenamed("_c0","Reviewer") \
    .withColumnRenamed("_c1","Item") \
    .withColumnRenamed("_c2","Rating") \
    .withColumnRenamed("_c3","Timestamp")

In [9]:
df_items.show(n=10, truncate=False)

+--------------+----------+------+----------+
|Reviewer      |Item      |Rating|Timestamp |
+--------------+----------+------+----------+
|A1NHVWB3UKEBNN|0143144723|4.0   |1271376000|
|A4ZKY17VUBPZH |B005X9LJBM|5.0   |1386028800|
|A1FTQLG098SXR9|B007YA15B2|5.0   |1342310400|
|A3QLVWSY7DFVJ4|1497408180|4.0   |1404864000|
|AM8D6ZN0IRO8K |B0077RUQJ4|5.0   |1381017600|
|A1CULHD7XSO6AZ|B0069G5Z9K|4.0   |1360454400|
|A12JBHOSYHW03 |B006Z6E8AG|5.0   |1363305600|
|A2F7F1VJZTZF0P|B003WETVKG|2.0   |1305676800|
|A2RLLV430OWYJC|B001GNCM2O|5.0   |1250380800|
|A1PFZN9J7XORUT|0062085484|3.0   |1367280000|
+--------------+----------+------+----------+
only showing top 10 rows



Check for Null or NaN values

In [10]:
df_items.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_items.columns]).show()

+--------+----+------+---------+
|Reviewer|Item|Rating|Timestamp|
+--------+----+------+---------+
|       0|   0|     0|        0|
+--------+----+------+---------+



Since there are no Null or NaN values we can safely convert "Rating" column to an Integer since Item ratings can only be natural numbers ranging from 1 to 5

In [11]:
df_items = df_items.withColumn("Rating", df_items["Rating"].cast(IntegerType()))

Check if .cast() was successful and look for values out of place

In [12]:
df_items.describe("Rating").show()

+-------+------------------+
|summary|            Rating|
+-------+------------------+
|  count|            826344|
|   mean| 4.162818390403997|
| stddev|1.2613187664122265|
|    min|                 1|
|    max|                 5|
+-------+------------------+



## Indexing

Two methods were tried since we were never able to run StringIndexer on more than 10% of the dataset

### PySpark Pipeline

In [ ]:
indexers = [StringIndexer(inputCol="Reviewer", outputCol="ReviewerID", handleInvalid="skip") , StringIndexer(inputCol="Item", outputCol="ItemID", handleInvalid="skip")]

pipeline = Pipeline(stages=indexers)
df_items_indexed = pipeline.fit(df_items).transform(df_items)

df_items_indexed.show()

### One column at a time

For some reason that allows StringIndexer to fit a greater sample than Pipeline before eventually Java runs out of memory or KryoSerializer throws a BufferOverflowing error

In [14]:
indexerItem = StringIndexer(inputCol="Item", outputCol="ItemID")
df_items_index_users = indexerItem.fit(df_items).transform(df_items)
df_items_index_users.select("Item","ItemID").show(truncate=False)

del df_items

indexerUsers = StringIndexer(inputCol="Reviewer", outputCol="ReviewerID")
df_items_indexed = indexerUsers.fit(df_items_index_users).transform(df_items_index_users)
df_items_indexed.select("Reviewer","ReviewerID").show(truncate=False)

del df_items_index_users

+----------+--------+
|Item      |ItemID  |
+----------+--------+
|0143144723|7360.0  |
|B005X9LJBM|421419.0|
|B007YA15B2|98824.0 |
|1497408180|13685.0 |
|B0077RUQJ4|436462.0|
|B0069G5Z9K|435.0   |
|B006Z6E8AG|11977.0 |
|B003WETVKG|369271.0|
|B001GNCM2O|317348.0|
|0062085484|2480.0  |
|B00BS85W88|46505.0 |
|0890897603|56565.0 |
|B001KZM4Y4|8386.0  |
|B008OBFL92|45160.0 |
|B0097GVJWQ|1396.0  |
|B00023B0A4|242814.0|
|B0049J4MVQ|379798.0|
|B008TLBY16|462272.0|
|B0047TNN7M|378830.0|
|B0063MI3LC|423825.0|
+----------+--------+
only showing top 20 rows

+--------------+----------+
|Reviewer      |ReviewerID|
+--------------+----------+
|A1NHVWB3UKEBNN|22659.0   |
|A4ZKY17VUBPZH |576158.0  |
|A1FTQLG098SXR9|137929.0  |
|A3QLVWSY7DFVJ4|543543.0  |
|AM8D6ZN0IRO8K |661154.0  |
|A1CULHD7XSO6AZ|19190.0   |
|A12JBHOSYHW03 |72939.0   |
|A2F7F1VJZTZF0P|31688.0   |
|A2RLLV430OWYJC|372323.0  |
|A1PFZN9J7XORUT|23321.0   |
|A364ZKFM1VLW8U|40549.0   |
|A8IY2M9ZW6VNQ |593544.0  |
|A26G1NEO9AK5XK|269023.0  

## Cleaning

Double check schema

In [15]:
df_items_indexed.printSchema()

root
 |-- Reviewer: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Timestamp: integer (nullable = true)
 |-- ItemID: double (nullable = false)
 |-- ReviewerID: double (nullable = false)



Drop "TimeStamp" column since we are not going to use it

In [16]:
df_items_indexed = df_items_indexed.drop("TimeStamp")

## Store data

Display 10 rules to make sure the DataFrame is as we want it

In [17]:
df_items_indexed.show(10, truncate=False)

+--------------+----------+------+--------+----------+
|Reviewer      |Item      |Rating|ItemID  |ReviewerID|
+--------------+----------+------+--------+----------+
|A1NHVWB3UKEBNN|0143144723|4     |7360.0  |22659.0   |
|A4ZKY17VUBPZH |B005X9LJBM|5     |421419.0|576158.0  |
|A1FTQLG098SXR9|B007YA15B2|5     |98824.0 |137929.0  |
|A3QLVWSY7DFVJ4|1497408180|4     |13685.0 |543543.0  |
|AM8D6ZN0IRO8K |B0077RUQJ4|5     |436462.0|661154.0  |
|A1CULHD7XSO6AZ|B0069G5Z9K|4     |435.0   |19190.0   |
|A12JBHOSYHW03 |B006Z6E8AG|5     |11977.0 |72939.0   |
|A2F7F1VJZTZF0P|B003WETVKG|2     |369271.0|31688.0   |
|A2RLLV430OWYJC|B001GNCM2O|5     |317348.0|372323.0  |
|A1PFZN9J7XORUT|0062085484|3     |2480.0  |23321.0   |
+--------------+----------+------+--------+----------+
only showing top 10 rows



Save Dataframe do parquet format to use with SparkSQL

In [18]:
output_items = "data.parquet"
df_items_indexed.write.mode("overwrite").parquet(output_items)

In [19]:
df_items_indexed.write.mode("overwrite").saveAsTable("DataTable")